<a href="https://colab.research.google.com/github/hodaka/MakeTrimmingMap/blob/work/%E8%87%AA%E5%8B%95%E3%83%88%E3%83%AA%E3%83%9F%E3%83%B3%E3%82%B0_%E3%83%95%E3%82%A9%E3%83%AB%E3%83%80%E6%8C%87%E5%AE%9A%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 人物を切り抜く準備としてのトリミングマップを作製するためのスクリプト


---
[参考URL](https://v-hramchenko.medium.com/how-to-cut-out-a-person-in-an-image-with-open-source-projects-e5e7f8798d5c)

1.   各種ライブラリのインポート

In [2]:
# GPUサポートを備えたTensorライブラリ(PyTorch自体はインストール済みcolabにインストール済み？)
import torch
# ファイル操作用
import os
# 画像操作ライブラリ
import cv2
# ファイルパス操作用ライブラリ
import glob
# 機械学習用拡張ライブラリ
import numpy as np
# 計算結果をグラフィカルに表示するライブラリ
import matplotlib.pyplot as plt

# セグメンテーション用ライブラリ
from torchvision.models.segmentation import deeplabv3_resnet101
# 画像の前処理や学習済みモデルなどを提供するエコシステム
from torchvision import transforms
# ファイル操作関連ライブラリ
from google.colab import files
# デバック用画像表示用ライブラリ
from IPython.display import Image, display

# 輪郭切り抜き用ライブラリ
!git clone https://github.com/MarcoForte/FBA-Matting.git
%cd FBA-Matting

import sys
sys.path.append("./FBA_Matting")

from demo import np_to_torch, pred, scale_input
from dataloader import read_image, read_trimap
from networks.models import build_model

class Args:
    encoder = 'resnet50_GN_WS'
    decoder = 'fba_decoder'
    weights = 'FBA.pth'

args=Args()
# 学習用のモデルを構築
try:
    model = build_model(args)
except:
    !gdown  https://drive.google.com/uc?id=1T_oiKDE_biWf2kqexMEN7ObWqtXAzbB1
    model = build_model(args)

Cloning into 'FBA-Matting'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 119 (delta 48), reused 47 (delta 10), pack-reused 0
Receiving objects: 100% (119/119), 4.59 MiB | 8.11 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/FBA-Matting
modifying input layer to accept 11 channels
Downloading...
From: https://drive.google.com/uc?id=1T_oiKDE_biWf2kqexMEN7ObWqtXAzbB1
To: /content/FBA-Matting/FBA.pth
139MB [00:01, 124MB/s] 
modifying input layer to accept 11 channels


2.   グーグルドライブをマウント



In [3]:
drive.mount('/drive')

Mounted at /drive


In [4]:
# 対象を取得するセグメンテーション用ライブラリを定義
def make_deeplab(device):
    deeplab = deeplabv3_resnet101(pretrained=True).to(device)
    deeplab.eval()
    return deeplab

device = torch.device("cpu")
deeplab = make_deeplab(device)




Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


In [ ]:

# 読み取り対象のフォルダパスを指定
read_path = "/drive/MyDrive/ImageData/NamioHarukawa_base"
# 出力対象のフォルダパスを指定
output_path = read_path + "/ResultOutput/"

#ディレクトリ作成(なければ)
if not os.path.exists(output_path):
  os.makedirs(output_path)

for idx,imageFilePath in enumerate(glob.glob(read_path + "/*.*")):
  print(imageFilePath)
  img_orig = cv2.imread(imageFilePath, 1)
  k = min(1.0, 1024/max(img_orig.shape[0], img_orig.shape[1]))
  img = cv2.resize(img_orig, None, fx=k, fy=k, interpolation=cv2.INTER_LANCZOS4)

  deeplab_preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  def apply_deeplab(deeplab, img, device):
    input_tensor = deeplab_preprocess(img)
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        output = deeplab(input_batch.to(device))['out'][0]
    output_predictions = output.argmax(0).cpu().numpy()
    # 15が人を表しているらしい…
    return (output_predictions == 15)

  mask = apply_deeplab(deeplab, img, device)
  
  trimap = np.zeros((mask.shape[0], mask.shape[1], 2))
  trimap[:, :, 1] = mask > 0
  trimap[:, :, 0] = mask == 0
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(25,25))

  trimap[:, :, 0] = cv2.erode(trimap[:, :, 0], kernel)
  trimap[:, :, 1] = cv2.erode(trimap[:, :, 1], kernel)

  fg, bg, alpha = pred((img/255.0)[:, :, ::-1], trimap, model)

  img_ = img_orig.astype(np.float32)/255
  alpha_ = cv2.resize(alpha, (img_.shape[1], img_.shape[0]), cv2.INTER_LANCZOS4)
  fg_alpha = np.concatenate([img_, alpha_[:, :, np.newaxis]], axis=2)
  outPath = output_path + "trimed_"+ str(idx) + ".png"
  print(outPath)
  # PNGファイルとして保存しないと透過設定が有効にならない？
  cv2.imwrite(outPath, (fg_alpha*255).astype(np.uint8))

/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa119.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_0.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa115.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_1.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa099.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_2.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa066.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_3.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa062.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_4.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa054.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_5.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHarukawa053.jpg
/drive/MyDrive/ImageData/NamioHarukawa_base/ResultOutput/trimed_6.png
/drive/MyDrive/ImageData/NamioHarukawa_base/NamioHaruka